****GRADED CHALLENGE 3****

Galih Sukmanaputra  
HCK 005

Latar Belakang :    
Platform belanja online saat ini sangat banyak digemari oleh masyarakat luas dari berbagai kalangan karena dianggap praktis dan efisien terhadap waktu dan tenaga yang dikeluarkan. Sebagai seorang data analyst di The Look E-commerce saya harus membuat laporan evaluasi aktivitas penjualan di platform tersebut. Untuk memberikan gambaran kepada tim marketing agar keuntungan perusahaan semakin meningkat.

Tujuan dari tugas ini adalah :    
1. Memahami daya tarik customer terhadap suatu barang di dalam e-commerce The Look. 
2. Mencari tahu barang yang berpotensi terjual dengan keuntungan yang tinggi dalam rentang waktu tertentu. 
3. Memberikan rekomendasi kepada tim marketing dari The Look terkait produk apa saja yang penjualannya harus diprioritaskan


Problem Statement :  
Perusahaan The Look menghadapi tantangan dalam menentukan segmentasi pasar terkait kategori produk apa saja yang paling laris dijual  dan menghasilkan keuntungan paling tinggi di tahun 2022 pada kuartal tertentu sehingga dibutuhkan analisis ketertarikan pelanggan terhadap produk yang ditawarkan. Berikut ini **poin penjabaran** analisis yang mengacu pada persoalan perusahaan


In [1]:
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

project_id = "deft-scope-381907" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


Menampilkan tabel

In [ ]:

import pandas as pd

df = client.query('''

SELECT * 
FROM `bigquery-public-data.thelook_ecommerce.orders`

''').to_dataframe()
df

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,11,15,Shipped,F,2022-12-06 15:07:00+00:00,NaT,2022-12-08 12:34:00+00:00,NaT,1
1,14,20,Shipped,F,2021-01-19 01:34:00+00:00,NaT,2021-01-19 21:58:00+00:00,NaT,1
2,23,27,Shipped,F,2021-03-29 16:32:00+00:00,NaT,2021-04-01 06:28:00+00:00,NaT,1
3,25,27,Shipped,F,2020-11-15 16:32:00+00:00,NaT,2020-11-17 07:05:00+00:00,NaT,1
4,43,42,Shipped,F,2023-03-13 10:15:00+00:00,NaT,2023-03-15 05:40:00+00:00,NaT,1
...,...,...,...,...,...,...,...,...,...
124653,124620,99970,Processing,M,2022-02-14 05:33:00+00:00,NaT,NaT,NaT,1
124654,124633,99979,Processing,M,2023-04-14 04:59:18.097600+00:00,NaT,NaT,NaT,1
124655,124634,99979,Processing,M,2023-04-14 04:59:18.097600+00:00,NaT,NaT,NaT,2
124656,124656,99998,Processing,M,2023-02-24 16:15:00+00:00,NaT,NaT,NaT,2


1. Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?

In [ ]:
client.query('''
SELECT 
    Q,
    month,  
    COUNT(order_id) AS total_transaction
  FROM
    (SELECT *, 
        status,
        EXTRACT(YEAR FROM created_at) AS year,
        EXTRACT(MONTH FROM created_at) AS month,
        EXTRACT(QUARTER FROM created_at) AS Q
    FROM `bigquery-public-data.thelook_ecommerce.orders`
    WHERE status = 'Complete')
    WHERE year = 2022 and Q between 1 and 3
    group by month, Q
    order by Q

''').to_dataframe()

,Q,month,total_transaction
0,1,1,717
1,1,2,704
2,1,3,768
3,2,6,976
4,2,4,865
5,2,5,908
6,3,9,1185
7,3,7,1070
8,3,8,1144


Jawaban : Jika dilihat dari hasil filter data yang dilakukan, diketahui bahwa selama tahun 2022 dalam jangka waktu antara Q1 dan Q3 jumlah transaksi semakin meningkat setiap bulannya. Total jumlah transaksi terbanyak terjadi pada bulan ke 9 (september) dengan jumlah sebanyak 1185 transaksi per quartal ke 3.

2. Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?

In [ ]:
client.query('''
SELECT 
    Q,
    month,  
    sum(sale_price) AS total_USD
  FROM
    (SELECT *, 
        status,
        EXTRACT(YEAR FROM created_at) AS year,
        EXTRACT(MONTH FROM created_at) AS month,
        EXTRACT(QUARTER FROM created_at) AS Q
    FROM `bigquery-public-data.thelook_ecommerce.order_items`
    WHERE status = 'Complete')
    WHERE year = 2022 and Q between 1 and 3
    group by month, Q
    order by month

''').to_dataframe()

,Q,month,total_USD
0,1,1,63376.380109
1,1,2,61518.120096
2,1,3,68069.050040
3,2,4,73546.890092
4,2,5,79337.690119
5,2,6,87005.690090
6,3,7,91475.250053
7,3,8,101345.860173
8,3,9,104123.500200


Jawaban : Jika dilihat dari tabel yang dihasilkan total penjualan dalam USD terbanyak terjadi pada bulan 8 (agustus) 2022.

3. Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?

In [ ]:
client.query('''
SELECT 
    Q,
    month,  
    COUNT(user_id) AS total_user
  FROM
    (SELECT *, 
        status,
        EXTRACT(YEAR FROM created_at) AS year,
        EXTRACT(MONTH FROM created_at) AS month,
        EXTRACT(QUARTER FROM created_at) AS Q
    FROM `bigquery-public-data.thelook_ecommerce.order_items`
    WHERE status = 'Complete')
    WHERE year = 2022 and Q between 1 and 3
    group by month, Q
    order by month

''').to_dataframe()

,Q,month,total_user
0,1,1,1039
1,1,2,1008
2,1,3,1158
3,2,4,1232
4,2,5,1322
5,2,6,1435
6,3,7,1541
7,3,8,1678
8,3,9,1708


Jawaban : Berdasarkan tabel di atas diketahui bahwa tiap bulannya user yang melakukan transaksi rata-rata berjumlah 1000 user

4. Kategori produk apa saja yang paling banyak dibeli (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [ ]:
client.query('''
SELECT 
category,
top_product, 
month, 
total_category,
Q
    FROM (SELECT RANK() OVER(PARTITION BY month ORDER BY total_category DESC) AS top_product, 
          category,
          month, 
          total_category,
          Q
           FROM (SELECT COUNT(category) AS total_category, 
                 category, 
                 month,
                 Q, 
                FROM (SELECT 
                      products.category,
                      EXTRACT(year from created_at) AS year,
                      EXTRACT(month from created_at) AS month,
                      EXTRACT(quarter from created_at) AS Q
                      FROM `bigquery-public-data.thelook_ecommerce.order_items` AS order_items
                      JOIN `bigquery-public-data.thelook_ecommerce.products` AS products
                      ON products.id = order_items.product_id
                      WHERE status = 'Complete')
                WHERE year = 2022 AND Q between 1 AND 3
                GROUP BY month,category,Q
                ORDER BY month,total_category)  
        )     
WHERE top_product = 1
ORDER BY month ASC

''').to_dataframe()

,category,top_product,month,total_category,Q
0,Shorts,1,1,79,1
1,Swim,1,1,79,1
2,Jeans,1,2,79,1
3,Intimates,1,3,94,1
4,Fashion Hoodies & Sweatshirts,1,4,113,2
5,Intimates,1,5,96,2
6,Intimates,1,6,119,2
7,Tops & Tees,1,7,118,3
8,Jeans,1,8,131,3
9,Intimates,1,9,129,3


Jawaban : Kategori yang paling banyak dibeli selama tahun 2022 dalam rentang Q1 dan Q3 adalah kategori Intimates.

5. Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [ ]:
client.query('''
SELECT 
      top_product,
      category, 
      total_revenue,
      month,
      Q
FROM
      (SELECT RANK() OVER(PARTITION BY month ORDER BY total_category DESC) AS top_product, 
                  category, 
                  month, 
                  total_category,
                  total_revenue, 
                   Q
      FROM
            (SELECT COUNT(category) AS total_category, 
                        category, 
                        month,
                        Q,
                        sum(revenue) AS total_revenue, 
            FROM (SELECT 
                  products.retail_price,
                  products.cost, products.retail_price - products.cost AS revenue, 
                  products.category,
                  EXTRACT(year from created_at) AS year,
                  EXTRACT(month from created_at) AS month,
                  EXTRACT(quarter from created_at) AS Q
                  FROM `bigquery-public-data.thelook_ecommerce.order_items` AS order_items
                  JOIN `bigquery-public-data.thelook_ecommerce.products` AS products
                  ON products.id = order_items.product_id
                  WHERE status = 'Complete')
            WHERE year = 2022 and Q between 1 AND 3
            GROUP BY month,category,Q
            ORDER BY month ASC,total_revenue DESC)  
      )     
WHERE top_product = 1
ORDER BY month ASC
''').to_dataframe()

,top_product,category,total_revenue,month,Q
0,1,Shorts,1879.684617,1,1
1,1,Swim,2318.625931,1,1
2,1,Jeans,3520.013145,2,1
3,1,Intimates,1931.279146,3,1
4,1,Fashion Hoodies & Sweatshirts,2550.016811,4,2
5,1,Intimates,2044.690095,5,2
6,1,Intimates,1953.415066,6,2
7,1,Tops & Tees,1883.814928,7,3
8,1,Jeans,6193.654296,8,3
9,1,Intimates,2217.751663,9,3


Jawaban : Jumlah pendapatan selama tahun 2022 pada retang Q1 dan Q3 cenderung tidak stabil. Total pendapatan tertinggi terjadi pada bulan ke 8 (Agustus) 2022 dengan kategori jeans.

6. Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan? (Hint: Kamu bisa menggunakan korelasi)


In [ ]:
client.query('''
SELECT CORR(total_user, total_selling) AS corr1,
        CORR(total_user, total_transaction) AS corr2
FROM (
      SELECT month,
        COUNT(user_id) AS total_user,
        SUM(sale_price) AS total_selling,
        SUM (month) AS total_transaction
      FROM (
            SELECT DISTINCT user_id, 
                  created_at, 
                  status, 
                  sale_price,
            EXTRACT(month FROM created_at) AS month,
            FROM `bigquery-public-data.thelook_ecommerce.order_items`
            WHERE status = 'Complete' AND created_at BETWEEN '2022-01-01' AND '2022-09-30'
            ) 
      GROUP BY month
      ORDER BY month
      )
''').to_dataframe()

,corr1,corr2
0,0.997616,0.988752


Jawaban : Jika dilihat dari hasil perhitungan korelasi antara dua hubungan di atas diketahui bahwa korelasi terbesar terjadi antara jumlah customer dan total penjualan, dimana semakin tinggi total customer yang melakukan pembelian maka semakin tinggi juga total penjualan yang terjadi. 

POIN ANALISIS

Berikan kesimpulan dari laporan/informasi yang dibuat berdasarkan problem statement/poin penjabaran menggunakan bahasa awam! (boleh dalam beberapa paragraf)


Jawaban :    
Dari hasil analisis yang dilakukan dapat diperoleh kesimpulan bahwa perusahaan The Look sebaiknya meningkatkan penjualan dengan kategori **jeans** di bulan Agustus pada tahun 2023 berikutnya agar profit perusahaan dapat meningkat secara sinificant. Sementara untuk total banyaknya penjualan produk kategori **intimates** harus dipertahankan di angka 120 agar profit perusahaan tetap stabil.  

POIN PERTANYAAN

1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!

2. Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?
3. CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!


Jawaban :    
1. Ya, karena berdasarkan poin penjabaran yang dituangkan dalam permasalahan soal sudah mencakup kriteria SMART dan komponen 5W yang sebelumnya telah dijelaskan pada sesi lecture di minggu ketiga. 

2. Menurut saya kurang masuk akal, karena jika dijumlahkan, total pendapatan tiap bulan per kuartal hanya menyentuh angka menyentuh angka 20000. Hal ini mungkindapat terjadi jika perusahaan memperbanyak segmen kategori penjualan lainnya dan mungkin melakukan ekspansi jaringan lebih luas.

3. Berdasarkan analisis yang saya lakukan saya melihat bahwa trend kenaikan penjualan yang terjadi sepanjang kuartal 1 hingga 3 di tahun 2022 cukup meningkat. Peningkatan tersebut terlihat pada jumlah transaksi dan GMV yang meningkat. Meskipun secara keuntungan cenderung normal dan merata. Jika dilihat, prospek pada kuartal ke 4 kemungkinan akan semakin meningkat. Hal ini dapat dikaitkan dengan fenomena sosial dimana biasanya menjelang akhir tahun customer akan cenderung membeli barang atau pakaian tertentu untuk merayakan tahun baru. Dengan kata lain, perusahaan ini layak untuk mendapatkan investor baru.